In [26]:
# The following is a sequential workflow, for creating blog creation post

In [27]:
# Importing basic libraries
from google.adk.agents import Agent, SequentialAgent
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search
from google.adk.runners import InMemoryRunner
from dotenv import load_dotenv
import os

In [28]:
# Checking the configuration
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("API KEY FOUND")
else:
    print("API KEY NOT FOUND")

API KEY FOUND


In [29]:
# Setting up the retry configuration
retry_config = types.HttpRetryOptions(
    initial_delay=2, # Initial delay before geneating the response
    attempts=4, # Maximum number of attempts 
    exp_base=8,
    http_status_codes=[429, 500, 503, 504]
)

In [30]:
# Outline agent: Used to create the outline of the blog
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought """,
    output_key="blog_outline"
)

print("outline_agent created")

outline_agent created


In [31]:
# Writer agent: Writes the full blog post based on the outline provided as input from the outline agent
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction=''' Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-words blog post with an engaging and informative tone.''',
    output_key="blog_draft",
)
print("writer_agent created")

writer_agent created


In [32]:
# Editor agent: The goal of this agent is to edit and polish the draft provided by the writer agent
editor_agent=Agent(
    name="EditorAgent",
    model=Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config
    ),
    instruction='''Given the blog draft: {blog_draft},
    Your task is to edit the blog draft, polish the text by fixing the grammatical errors, improving the flow and sentence
    structure, thereby enhancing the overall quality
    ''',
    output_key="final_blog"
)

print("editor_agent created")

editor_agent created


In [33]:
# Root agent for managing the sequential workflow
root_agent=SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent]
)

print("sequential_agent created")

sequential_agent created


In [34]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post on efficiency of web-crypto api compared to bcrypt in production"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a blog post on efficiency of web-crypto api compared to bcrypt in production
OutlineAgent > ### Blog Post Outline: Web Crypto API vs. Bcrypt for Production Hashing: Is It Time for a Change?

**Catchy Headline:** Web Crypto API vs. Bcrypt: Which Hashing Powerhouse Reigns Supreme in Production?

**Introduction Hook:** In the ever-evolving landscape of web security, robust password hashing is non-negotiable. For years, Bcrypt has been the undisputed champion, a tried-and-true workhorse for protecting user credentials. However, a new contender has emerged: the Web Crypto API. But can this browser-native solution truly compete with the established Bcrypt in the demanding environment of production? Let's dive in and find out.

**Main Sections:**

**1. Understanding the Contenders: Bcrypt's Legacy and Web Crypto's Promise**

*   **Bcrypt: The Battle-Tested Standard:** Discuss Bcrypt's strengths, including its adaptive nature (work fact